## GAE 
1. To prove this, we collect all $r_i$'s and and all $v_{\omega}(s_i)$'s
$$
\begin{split}
(1-\lambda)\sum_{h=0}^{\inf}\lambda^h\delta_{t,h} &= (1-\lambda)\sum_{h=0}^{\inf}\lambda^h(\sum_{i=t}^{t+h}\gamma^{i-t}\cdot r_i + \gamma^{h+1}\cdot v_\omega(s_{t+h+1}) - v_{\omega}(s_t))\\
& = (1-\lambda)\sum_{h=0}^{\inf}(\sum_{i=t}^{t+h}\lambda^h\gamma^{i-t}\cdot r_i) + (1-\lambda)\sum_{h=0}^{\inf}(\gamma^{h+1}\cdot v_\omega(s_{t+h+1}) - v_{\omega}(s_t))\\
& = (1-\lambda)(\gamma^0\sum_{i=0}^{\inf}\lambda^i r_{t} + \gamma^{1}\sum_{i=1}^{\inf}\lambda^i r_{t+1} + \gamma^{2}\sum_{i=2}^{\inf}\lambda^i r_{t+2} + \cdots + \gamma^{h}\sum_{i=h}^{\inf}\lambda^i r_{t+h})+ \sum_{h=0}^{\inf}(1-\lambda)\lambda^h\gamma^{h+1}\cdot v_\omega(s_{t+h+1}) - v_{\omega}(s_t)\\
& = (1-\lambda)\sum_{h=0}^{\inf}\frac{(\gamma\lambda)^h}{1-\lambda}r_t^{h} + \sum_{h=0}^{\inf}(\lambda^h-\lambda^{h+1})\gamma^{h+1}\cdot v_\omega(s_{t+h+1})- v_{\omega}(s_t)\\
& = (1-\lambda)\sum_{h=0}^{\inf}\frac{(\gamma\lambda)^h}{1-\lambda}r_t^{h} + \sum_{h=0}^{\inf}\lambda^h\gamma^{h}\cdot (\gamma v_\omega(s_{t+h+1}) - v_\omega(s_{t+h}))\\
& = \sum_{h=0}^{\inf}(\gamma\lambda)^h\delta_{t+h,0}
\end{split}
$$
2. The role of $\lambda$ is the weight given to each advantage estimator of the $Q$ function. In this case, the longer one has to expand, the less weight is given to the advantage estimator. The role of $\gamma$ is to discount the reward at further timestep as they are less certain.